In [1]:
import boto3
from botocore.exceptions import ClientError

ERROR_HELP_STRINGS = {
    # Common Errors
    'InternalServerError': 'Internal Server Error, generally safe to retry with exponential back-off',
    'ProvisionedThroughputExceededException': 'Request rate is too high. If you\'re using a custom retry strategy make sure to retry with exponential back-off.' +
                                              'Otherwise consider reducing frequency of requests or increasing provisioned capacity for your table or secondary index',
    'ResourceNotFoundException': 'One of the tables was not found, verify table exists before retrying',
    'ServiceUnavailable': 'Had trouble reaching DynamoDB. generally safe to retry with exponential back-off',
    'ThrottlingException': 'Request denied due to throttling, generally safe to retry with exponential back-off',
    'UnrecognizedClientException': 'The request signature is incorrect most likely due to an invalid AWS access key ID or secret key, fix before retrying',
    'ValidationException': 'The input fails to satisfy the constraints specified by DynamoDB, fix input before retrying',
    'RequestLimitExceeded': 'Throughput exceeds the current throughput limit for your account, increase account level throughput before retrying',
}


In [5]:
def create_dynamodb_client():
    return boto3.client(
        "dynamodb", 
        region_name="localhost", 
        endpoint_url="http://localhost:8000", 
        aws_access_key_id="vrolr", 
        aws_secret_access_key="l36sz"
        )

def create_query_input():
    return {
        "TableName": "OnlineShop",
        "KeyConditionExpression": "#cd420 = :cd420 And #cd421 = :cd421",
        "ExpressionAttributeNames": {"#cd420":"PK","#cd421":"SK"},
        "ExpressionAttributeValues": {":cd420": {"S":"c#12345"},":cd421": {"S":"c#12345"}}
    }


def execute_query(dynamodb_client, input):
    try:
        response = dynamodb_client.query(**input)
        print(response)
        # Handle response
    except ClientError as error:
        handle_error(error)
    except BaseException as error:
        print("Unknown error while querying: " + error.response['Error']['Message'])


def handle_error(error):
    error_code = error.response['Error']['Code']
    error_message = error.response['Error']['Message']

    error_help_string = ERROR_HELP_STRINGS[error_code]

    print('[{error_code}] {help_string}. Error message: {error_message}'
          .format(error_code=error_code,
                  help_string=error_help_string,
                  error_message=error_message))


def main():
    # Create the DynamoDB Client with the region you want
    dynamodb_client = create_dynamodb_client()

    # Create the dictionary containing arguments for query call
    query_input = create_query_input()

    # Call DynamoDB's query API
    execute_query(dynamodb_client, query_input)


if __name__ == "__main__":
    main()
    

{'Items': [{'SK': {'S': 'c#12345'}, 'EntityType': {'S': 'customer'}, 'PK': {'S': 'c#12345'}, 'Email': {'S': 'samaneh@example.com'}, 'Name': {'S': 'Samaneh'}}], 'Count': 1, 'ScannedCount': 1, 'ResponseMetadata': {'RequestId': '2f55df50-eae4-48e4-afae-2da5ff2e36a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 15 Apr 2024 09:03:00 GMT', 'x-amzn-requestid': '2f55df50-eae4-48e4-afae-2da5ff2e36a4', 'content-type': 'application/x-amz-json-1.0', 'x-amz-crc32': '2101062283', 'content-length': '171', 'server': 'Jetty(11.0.17)'}, 'RetryAttempts': 0}}
